In [2]:
import rasterio as rio
from rasterio.windows import Window
from rasterio.fill import fillnodata
from rasterio.shutil import copyfiles
import numpy as np
import glob
from tqdm import tqdm 
import re
import os

In [3]:
elev_files = glob.glob('/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/*/*levation_*.tif')
slope_files = glob.glob('/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/*/*lope_*.tif')
veg_files = glob.glob('/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/*/*egetation_*.tif')

all_files = elev_files + slope_files + veg_files

In [5]:
# this code is directly lifted from the block_corner_generator and hru_block_generator functions, in the 
# distributed raster reclass tool
def read_window_generator(hru_numrows,hru_numcols,blocksize):
    colrange = range(0,hru_numcols,blocksize)
    rowrange = range(0,hru_numrows,blocksize)
    for col in colrange:
        for row in rowrange:
            topleft = [col, row]
            
            if col == max(colrange):
                br_col = hru_numcols
                block_width = hru_numcols - max(colrange)
            else:
                br_col = col + blocksize
                block_width = blocksize
            if row == max(rowrange):
                br_row = hru_numrows
                block_height = hru_numrows - max(rowrange)
            else:
                br_row = row + blocksize
                block_height = blocksize
                
            bottomright = [br_col, br_row]
            
            col_offset = col
            row_offset = row
            
            read_window =  Window(col_offset, row_offset, block_width, block_height)
            
            yield(read_window)

In [6]:
def patch_nodata(filepath):
    
    basename = os.path.basename(filepath)

    writefolder = '/media/eric/data/data/auckland/hru_creation/nodata_interpolation'
    writepath = os.path.join(writefolder,basename)

    copyfiles(filepath,writepath)

    with rio.open(writepath) as src:
        hru_numrows, hru_numcols = src.shape

    read_windows = read_window_generator(hru_numrows, hru_numcols, 5000)

    for read_window in read_windows:

        with rio.open(writepath) as src:
            rast_block = src.read(1, window=read_window)
            out_meta = src.meta.copy()

        mask = rast_block != -1
        rast_block = fillnodata(rast_block,mask,max_search_distance=10)

        with rio.open(writepath, "r+", **out_meta) as dst:
            dst.write(rast_block,window=read_window,indexes=1)

In [14]:
filepath = elev_files[5]
patch_nodata(filepath)

In [11]:
for filepath in tqdm(all_files[25:]):
    patch_nodata(filepath)

100%|██████████| 4/4 [02:23<00:00, 35.77s/it]


In [6]:
filepath

'/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers/Wairoa/WithNoData/elevation_Wairoa.tif.tif'

In [11]:
with rio.open(filepath) as src:
    hru_numrows, hru_numcols = src.shape

In [4]:
filepath = all_files[0]

In [11]:
basename = os.path.basename(filepath)

writefolder = '/media/eric/data/data/auckland/hru_creation/nodata_interpolation'
writepath = os.path.join(writefolder,basename)

copyfiles(filepath,writepath)

with rio.open(writepath) as src:
    hru_numrows, hru_numcols = src.shape

read_windows = read_window_generator(hru_numrows, hru_numcols, 5000)

for read_window in read_windows:

    with rio.open(writepath) as src:
        rast_block = src.read(1, window=read_window)
        out_meta = src.meta.copy()

    mask = rast_block != -1
    rast_block = fillnodata(rast_block,mask,max_search_distance=10)

    with rio.open(writepath, "r+", **out_meta) as dst:
        dst.write(rast_block,window=read_window,indexes=1)

12it [00:14,  1.17s/it]


In [1]:
import geopandas as gpd

In [2]:
gpd.read_file('/media/eric/data/box/Projects/Auckland/5. Model/LSPC/ZonalRasterStatistics/HRU_Sliders/Raster_Layers_Combine/Tamaki/Combine.tif.vat.dbf')

,Value,Count,catchments,RLU,soil,zoning,landuseVeg,geometry
0,1,7512.0,3679,999,8,5,842,None
1,2,40741.0,3679,999,8,3,842,None
2,3,16531.0,3679,999,8,3,742,None
3,4,4014.0,3679,999,8,5,742,None
4,5,22513.0,3679,999,8,3,21,None
...,...,...,...,...,...,...,...,...
9096,9097,41.0,3724,170524,4,7,843,None
9097,9098,62.0,3724,170524,4,7,743,None
9098,9099,31.0,3709,170518,4,7,843,None
9099,9100,1.0,3724,170525,4,7,844,None
